# cutting tool for a worm gear

1. idea 1:  

<img src="../../docs/computing a profile_from_a_given_rack.jpg">

In [1]:
import sympy as sym
import numpy as np
from pygears.transformation import symbolic_transformation, numeric_transformation

In [5]:
t = sym.Symbol("t")
T1 = symbolic_transformation(np.pi / 2.,
                             np.array([1., 0., 0.]),
                             np.array([12.5,0., 1.15]))
T2 = symbolic_transformation(-t / 7.5,
                             np.array([0., 0., 1.]),
                             np.array([0., 0., 0.]))
T3 = symbolic_transformation(0.,
                             np.array([1., 0., 0.]),
                             np.array([0., 0., t]))

T = sym.nsimplify(T2.inv() @ T1.inv() @ T3, tolerance=10e-16)

In [7]:
T_fn = sym.lambdify(t, T)
dT_fn = sym.lambdify(t, T.diff(t))
dT_fn(0.)

array([[ 0.        ,  0.        , -0.13333333,  0.15333333],
       [-0.13333333,  0.        ,  0.        ,  0.66666667],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

Diese Methode funktioniert nicht, weil die Bedingung zur Bestimmung des Kontaktpunktes falsch ist.
Eine Bedingung für die generierung einer konstanten Übersetzung ist, dass die Normale auf die Kontaktfläche (Zahnstange) immer durch den Punkt p (Eingriffspunkt für Ersatzzahnrad (Zylinder) und Ersatzzahnstange (Quader) gehen muss.
Gesucht sind also Punkte auf der Fläche S welche verbunden mit P normal auf die Fläche stehen. Dies kann auch als minimaler Abstand von P zur Fläche gesehen werden.

für jedes u: min(norm(S(u,v)-P)) -> d(norm(S(u,v)-P)/dv = 0
die Änderung des Abstands ist 0 -> die Gerade steht normal auf die Fläche


## Vorgehensweise

1. approximate the surface by a BSplineSurface

- Erstellen eines "Cross-Sektion" objekts aus dem "Werkzeug"

<img src="cross_section.png">  

- Draft downgrade um Kanten zu bekommen
das Cross-section Objekt beinhaltet nicht die anfangs und end Kanten. Diese müssen zusätzlich vom Werkzeug extrahiert werden

- Part Loft zum erstellen einer schönen BSplinefläche

<img src="loft_bspline_flaechen.png"> 

3. Loft -> Surface

```python
# select the cutting faces
face_1 = App.ActiveDocument.Loft.Shape.Faces[0].copy()
face_2 = App.ActiveDocument.Loft001.Shape.Faces[0].copy()

# compute the contact curve:
bsp_1 = face_1.Surface
bsp_2 = face_2.Surface

```

4. Minimierung des Abstands zum "Pitch-Punkt"

```python
import scipy as scp
point = App.Vector(5., 0., 1.15 - time) 
xyz_1 = []
for v in np.linspace(0, 1, 5):
        def dist_1(u):
            distance = bsp_1.value(u, v) - point
            return distance.x ** 2 + distance.z ** 2
        u_1 = scp.optimize.minimize(dist_1, 0.5, tol=1e-6).x[0]
        xyz_1.append(bsp_1.value(u_1, v))
```

5. erstellen einer B-Spline Kurve welche durch die Kinematik T transformiert wird

```python
c_1 = part.BSplineCurve()
c_1.interpolate(Points=xyz_1)
c_1 = c_1.toShape()

part.show(c_1.transformShape(T))
```

6. Loft anwenden auf die erstellten BSpline Kurven

<img src="loft_of_generated_bsplines.png">

7. Array für das Zahnrad

<img src="gear_assembly.png">

In [9]:
import sympy as sym
import numpy as np
import scipy as sp

from pygears.transformation import symbolic_transformation, numeric_transformation

t, x, z, m = sym.symbols(["t", "x", "z", "m"], real=True)
s, alpha, n_t, y = sym.symbols(["s", "alpha", "n_t", "y"], real=True, positiv=True)

In [10]:
T_spiral = symbolic_transformation(t * 2 *  sym.pi, np.array([0, 0, 1]), np.array([0, 0, m * sym.pi * n_t * t ]))
T_spiral

Matrix([
[ cos(2*pi*t), sin(2*pi*t), 0,          0],
[-sin(2*pi*t), cos(2*pi*t), 0,          0],
[           0,           0, 1, pi*m*n_t*t],
[           0,           0, 0,        1.0]])

In [11]:
l = sym.Matrix([0, s * sym.cos(alpha), s * sym.sin(alpha), 1])
l

Matrix([
[           0],
[s*cos(alpha)],
[s*sin(alpha)],
[           1]])

In [12]:
spiral = (T_spiral @ l)
spiral

Matrix([
[ s*sin(2*pi*t)*cos(alpha)],
[ s*cos(alpha)*cos(2*pi*t)],
[pi*m*n_t*t + s*sin(alpha)],
[                      1.0]])

In [13]:
x_cross_section = sym.simplify(sym.solve(spiral[0] - x, t)[1])
x_cross_section  # parameter s

asin(x/(s*cos(alpha)))/(2*pi)

In [14]:
spiral_x = sym.simplify(spiral.subs({t: x_cross_section}))
spiral_x

Matrix([
[                                               x],
[s*sqrt(1 - x**2/(s**2*cos(alpha)**2))*cos(alpha)],
[   m*n_t*asin(x/(s*cos(alpha)))/2 + s*sin(alpha)],
[                                             1.0]])

In [15]:
y_cross_section = sym.simplify(sym.solve(spiral_x[1]- y, s)[0])
y_cross_section

sqrt(x**2 + y**2)/cos(alpha)

In [16]:
f_r = sym.Function("r")(x, y)
spiral_xy = sym.simplify(spiral_x.subs({s: y_cross_section}))
spiral_xy = spiral_xy.subs({sym.Abs(y): y, sym.sqrt(x**2 + y**2): f_r})
spiral_xy

Matrix([
[                                           x],
[                                           y],
[m*n_t*asin(x/r(x, y))/2 + r(x, y)*tan(alpha)],
[                                         1.0]])

In [17]:
z = sym.Function("z")(x, y)
z

z(x, y)

In [18]:
y_p =sym.Symbol("y_p")
dist_p = sym.sqrt(z**2 + (y - y_p)**2)
dist_p.diff(y)

(y - y_p + z(x, y)*Derivative(z(x, y), y))/sqrt((y - y_p)**2 + z(x, y)**2)

## for x = 0, for a first guess of t

In [1]:
import scipy as sp
import numpy as np
from freecad import part
from freecad import app
from pygears.transformation import numeric_transformation

debug = False
def compute_involute(module=1, teeth=15, height=5, worm_pitch_diameter=10, num_threads=1, alpha=np.deg2rad(20)):
    x = 0.
    r_w = module * teeth / 2
    y_p = worm_pitch_diameter / 2
    r_thales = r_w / 2.
    y_thales = y_p + r_thales
    
    def length(y):
        return (x**2 + y**2)**(0.5)
    
    def dlength_dy(y):
        return y / length(y)
    
    def z(y, t):
        r = length(y)
        return - module * num_threads * np.arcsin(x / r) / 2 + r * np.tan(alpha) + t
    
    def dz_dy(y):
        r = length(y)
        return module * num_threads * x * dlength_dy(y) / \
               (2 * np.sqrt(1 - x ** 2 / r ** 2 ) * r ** 2) + \
               np.tan(alpha) * dlength_dy(y)
        
    def distance_yp(y, t):
        return np.sqrt((y_p - y) ** 2 + z(y, t) ** 2)

    def distance_yp_2(y, t):
        return (y_p - y) ** 2 + z(y, t) ** 2
    
    def ddistance_yp_dy(y, t):
        return (y - y_p + z(y, t) * dz_dy(y)) / distance_yp(y, t)
    
    def distance_ythales(y, t):
        return np.sqrt((y_thales - y) ** 2 + z(y, t) ** 2)
    
    def min_ground(pars):
        y, t = pars
        # return the normal-condition + the intesection of the tooth-flank with the thales circle
        return ddistance_yp_dy(y, t) ** 2 + (distance_ythales(y, t) - r_thales) ** 2

    def min_head(pars):
        y, t = pars
        r_0 = y_p -  5 * module # * (1 + clearence)
        # y_inner = r_0 * np.cos(np.arcsin(x / r_0))
        return ddistance_yp_dy(y, t) ** 2 + (y - r_0) ** 2

    def analytic_solution_for_x_0():
        t = - (r_w + y_p) * np.tan(alpha)
        y = y_p + r_w * np.sin(alpha) ** 2
        return np.array([y, t])
    
    start = analytic_solution_for_x_0()
    if debug:
        print(f"analytic solution:   {analytic_solution_for_x_0()}")
        print(f"min_ground analytic: {min_ground(start)}")
        print(f"thales analytic:     {distance_ythales(start[0], start[1]) - r_thales}")
        print(f"normal analytic:     {ddistance_yp_dy(start[0], start[1])}")
        print()

    # t_end is once computed for x=0
    t_end = sp.optimize.minimize(min_head, [y_p, 0.]).x[1]

    xyz = []
    for x in np.linspace(-height / 2, height / 2, 20):
        xyz_section = []
        t_start = sp.optimize.minimize(min_ground, start).x[1]
        for t in np.linspace(t_start, t_end, 20):

            # compute the time (t) dependent transformation
            phi = np.pi / 2
            phi += y_p * np.tan(alpha) / r_w
            phi += - np.sign(alpha) * module * np.pi / 4. / r_w
            phi += t / r_w
            T_0 = numeric_transformation(phi, np.array([0., 0., 1.]))
            T_1 = numeric_transformation(-np.pi / 2, np.array([0., 1., 0.]))
            T_2 = numeric_transformation(0., np.array([0., 0., 1.]), np.array([0, y_p + r_w, 0.]))
            T = np.linalg.inv(T_2 @ T_1 @ T_0)
            
            # find point on curve for given t
            y = sp.optimize.root(ddistance_yp_dy, y_p, (t)).x[0]
            z_i = z(y, t) #  - y_p * np.tan(alpha) + np.sign(alpha) * module * np.pi / 4
            point = np.array([x, y, z_i, 1.])
            xyz_section.append((T @ point)[:3])
        xyz.append(np.array(xyz_section))

    return np.array(xyz)

# parameters
module = 1.
teeth = 50
height = 5
worm_pitch_diameter = 10
num_threads = 1
alpha = np.deg2rad(20)
y_p = worm_pitch_diameter / 2
r_w = teeth * module / 2
clearence = 0.25
head = 0.
    
# create two surfaces one for positive alpha and one for negative alpha
for alpha_i in [-alpha, alpha]:   
    curves = []
    xyz = compute_involute(
        module=module,
        teeth=teeth, 
        height=height,
        worm_pitch_diameter=worm_pitch_diameter,
        num_threads=num_threads,
        alpha=alpha_i)
    
    for line in xyz.transpose(1, 0, 2):
        bs = part.BSplineCurve()
        points = [app.Vector(*point) for point in line]
        bs.interpolate(points)
        curves.append(bs.toShape())
    part.show(part.makeLoft(curves))

# create cutting surfaces for head and bottom
r_head = y_p - module * (1 + head)
r_foot = y_p + module * (1 + clearence)

phi_head = np.arcsin(height / 2 / r_head)  # from + phi to - phi
phi_foot = np.arcsin(height / 2 / r_foot)

line_head = []
for phi_i in np.linspace(-phi_head, phi_head, 10):
    x = r_w + y_p - np.cos(phi_i) * r_head
    z = np.sin(phi_i) * r_head
    line_head.append([x, 0., z, 1])

line_foot = []
for phi_i in np.linspace(-phi_head, phi_head, 10):
    x = r_w + y_p - np.cos(phi_i) * r_foot
    z = np.sin(phi_i) * r_foot
    line_foot.append([x, 0., z, 1])

curves_head = []
curves_foot = []
phi_j_max = 2 * np.pi / teeth
for phi_j in np.linspace(-phi_j_max, phi_j_max, 10):
    T = numeric_transformation(phi_j, np.array([0., 0., 1.]))
    temp_points_foot = [app.Vector(*(T @ point)[:3]) for point in line_foot]
    temp_points_head = [app.Vector(*(T @ point)[:3]) for point in line_head]
    bsp_foot = part.BSplineCurve()
    bsp_head = part.BSplineCurve()
    bsp_foot.interpolate(temp_points_foot)
    bsp_head.interpolate(temp_points_head)
    curves_foot.append(bsp_foot.toShape())
    curves_head.append(bsp_head.toShape())

part.show(part.makeLoft(curves_foot))
part.show(part.makeLoft(curves_head))

PATH_TO_FREECAD_LIBDIR not specified, using default FreeCAD version in /Users/lo/mambaforge/envs/freecad/lib


<Part::PartFeature>